In [2]:
import pandas as pd
from openai import OpenAI
import threading
from queue import Queue
import traceback
import json
import time
import numpy as np

In [2]:
client = OpenAI(
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key = "sk-c6b10e9324b04e17a9156623061e6765"
)
# client = OpenAI(
#     base_url = "http://localhost:11434/v1",
#     api_key = "ollama"
# )

## 长评

In [4]:
long = pd.read_excel("./采集/长评.xlsx")
long

,电视剧名称,电视剧id,影评id,影评人名称,影评时间,评分,标题,有用数,无用数,影评内容
0,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...
1,﻿无尽的尽头,36656706,16688684,尘飞扬,2025-04-28 20:53:32,40.0,《无尽的尽头》：不是为罪行开脱，而是看清罪恶的土壤,139.0,5.0,一口气刷完8集《无尽的尽头》，我会继续追下去。 第五集，白恩宇对检察官郑雁来说的话，可谓掷地...
2,﻿无尽的尽头,36656706,16682513,比万大魔王,2025-04-25 19:18:04,30.0,可惜，痛点太明显,123.0,13.0,《无尽的尽头》聚焦的是未成年人犯罪。 出发点是好的，但就完成度来说，差了些。至少目前首更这几...
3,﻿无尽的尽头,36656706,16692186,薇羅尼卡,2025-04-30 18:16:43,40.0,在破碎中，看见坚持的人性光,53.0,6.0,说实话，点开《无尽的尽头》之前，我并没有抱特别大的期待。未成年人犯罪、检察官题材，这样的标签...
4,﻿无尽的尽头,36656706,16685028,唐山,2025-04-26 23:10:34,40.0,通过《无尽的尽头》，我们看到的是自己,80.0,8.0,如果曾被去年3·10邯郸三名未满14岁的初中生杀害同学案震惊；如果你知道，中国未成年犯罪率正...
...,...,...,...,...,...,...,...,...,...,...
16230,无所畏惧之永不放弃,36660843,16473510,娱娱鱼,2025-02-03 00:19:51,50.0,《无所畏惧2》观后感，许卓对梅大梁夫妇的恨，来自破产清算案,1.0,NaN,《无所畏惧2之永不放弃》观后感，我上篇怀疑许卓跟梅大梁夫妇有私人恩怨，结果是公家恩怨。 所有...
16231,无所畏惧之永不放弃,36660843,16771920,Pepper &amp; CC,2025-06-12 10:51:02,40.0,从心理学看第二季结局——李沅案是狗尾续貂，还是符合人设的大戏,NaN,NaN,假如没有李沅案，陈硕从玩世不恭到非罗英子不可，就是一个简单的爱情泡沫剧。李沅案整合了陈硕和罗...
16232,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...
16233,无所畏惧之永不放弃,36660843,16502090,娱娱鱼,2025-02-13 22:31:13,50.0,《无所畏惧2》结局，坏律师官司缠身，好律师财源广进,NaN,NaN,《无所畏惧2》结局，坏律师官司缠身，好律师财源广进。方丽虹和许卓因魏丞案而受到了反噬。 在罗...


In [145]:
len(long)

16235

In [4]:
def cleanReview(comment):
    jsonf = '{"clean":<这里填入清洗后的评论内容>}'
    prompt = f"""
我将提供一条用户的电视剧评论，请你判断其中是否包含与电视剧本身相关的评价信息，不论是本身的质量还是宣传，以及受众观看剧集是产生的感悟，都是对剧的评价。

请清洗掉以下内容：
- 与电视剧无关的聊天、广告、吐槽或无意义内容
- 无具体信息的感叹（如“哈哈哈哈哈”、“看了好几遍了”、“真的无语”）
- 对剧情细节的描述和引用，与评价无关
- 与其他作品、事件或话题无关联的比较
- 重复、冗余、空泛的文字

你的任务是：
1. 如果评论中包含有效的剧评内容，提取并保留这些部分。
2. 如果评论中完全没有剧评相关内容，请返回空字符串。
3. 不要过度发挥。

返回格式
以Json格式返回清洗后评论内容，请不要返回其他任何内容。具体格式为：
{jsonf}

请处理以下评论文本：
“{comment}”
"""
    response = client.chat.completions.create(
        # model="qwen3:32b",
        # model="qwen-plus-2025-04-28",
        # model="qwen-turbo-2025-04-28",
        # model="qwen-long-latest",
        model="qwen-long",
        messages=[
            {"role": "system", "content": "你是一个擅长文本清洗与信息提取的助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return result

In [6]:
def askLLM():
    while True:
        row = review_q.get()
        comment = row['影评内容']
        rid = row["影评id"]
        user = row["影评人名称"]
        dramma = row["电视剧名称"]
        try:
            res = cleanReview(comment)
            if res:
                write_q.put([rid, res])
                with print_lock:
                    print(dramma, user, '提问成功')
        except Exception as e:
            msg = f"提问失败 {dramma} {user} {rid}: {e}\n"
            # msg += traceback.format_exc()
            with print_lock:
                print(msg)
        review_q.task_done()
                

In [7]:
def writeRes():
    while True:
        rid, res = write_q.get()
        try:
            ans = json.loads(res)
            clean_comment = ans["clean"]
            with open('./clean/long.txt', "a", encoding="utf-8") as f:
                line = str(rid) + "\t" + clean_comment.replace("\t", "").replace("\n", "").strip() + "\n"
                f.write(line)
        except Exception as e:
            msg = f"存储失败 {rid}: {e}\n" 
            with print_lock:
                print(msg)
        write_q.task_done()

In [14]:
review_q = Queue()
write_q = Queue()
print_lock = threading.Lock()

for k in range(5):
    news_info_t = threading.Thread(target=askLLM)
    news_info_t.daemon = True
    news_info_t.start()
    time.sleep(5)
writing_thread = threading.Thread(target=writeRes)
writing_thread.daemon = True
writing_thread.start()

In [15]:
for i, row in long.iterrows():
    rid = row["影评id"]
    if str(rid) in list(already['评论id'].unique()):
        continue
    review_q.put(row)
review_q.join()
write_q.join()

提问失败 ﻿无尽的尽头 竹子 16776944: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-1ee3a013-b8cd-9d54-beaa-dc0c22577c4e', 'request_id': '1ee3a013-b8cd-9d54-beaa-dc0c22577c4e'}

提问失败 ﻿无尽的尽头 XXXX. 16721600: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-af3770a9-168d-9705-b18d-219486c51ecc', 'request_id': 'af3770a9-168d-9705-b18d-219486c51ecc'}

提问失败 ﻿无尽的尽头 魏二 16692562: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-0d099afa-5f97-9999-9195-8f8566cd774c', 'request_id': '0d099afa-5f97-9999-9195-8f8566cd774c'}

提问失败 ﻿无尽的尽头 双木读书 16714441: Error code: 400 - {'error': {'code': 'data_ins

In [155]:
already = []
with open("./clean/long.txt", encoding='utf-8') as f:
    for line in f:
        if not line:
            continue
        rid = line.split("\t", 1)[0]
        review = line.split("\t", 1)[-1].strip()
        already.append({'评论id':rid, '评论内容':review})
already = pd.DataFrame.from_records(already)
already

,评论id,评论内容
0,16703529,还有你们自己回去看看你们，办案的是谁？女检察官，她书记员是女的，副检察长是女的，一群那么优秀...
1,16688684,作为一部聚焦未成年人司法保护的悬疑剧，《无尽的尽头》不仅拍未成年人的犯罪事件，而且在拍未成年...
2,16682088,开集第一个案子很猛，对吧？因为这正是我们在媒体上常见的典型形象：那些令人憎恶的 “问题少年”...
3,16682513,《无尽的尽头》聚焦未成年人犯罪，出发点是好的，但完成度差了些。至少目前首更这几集的内容而言，...
4,16692477,聊聊《无尽的尽头》偷拍案。这一篇，可以说是头重脚轻，草草收场，离谱，莫名其妙。夏秋，她在整个...
...,...,...
16200,16747376,苏易水绝美，真的非常非常非常非常非常非常非常非常非常非常非常非常非常非常非常非常非常非常非常...
16201,16521023,先说结论：仙台有树，近五年最好仙侠。别说仙侠了，自信点，放整个古偶都是top1。先不说其他，...
16202,16500740,1）看的时候很快乐很安心，因为最近也经历跟女主类似的事情，看2的时候回头看1看着她冲冲冲仿佛...
16203,16504916,我很爱邓为饰演的苏易水，我能共情到他的痛苦，悔恨，思念，隐忍和爱。他打动了我。这是一部关于人...


In [156]:
already['状态'] = '已有'

In [157]:
already = already[~already['评论内容'].isna()]

In [158]:
already.to_csv('./clean/long_fix.csv', index=False, encoding='utf-8-sig')

## 短评

In [37]:
short = pd.read_excel('./采集/短评.xlsx')
short = short[~short["评论内容"].isna()].copy()
short

,电视剧名称,电视剧id,短评id,评论人,评论时间,评论点赞数,位置,评论内容,评分
0,﻿无尽的尽头,36656706,4520292443,反正都会后悔,2025-05-01 01:24:15,4449,广东,发现有些国人真挺好笑的，比如韩国一些现实题材通过电影影视呈现出来，就会说好厉害好敢拍，不像我...,40.0
1,﻿无尽的尽头,36656706,4515142798,小熊饼干脆脆面,2025-04-24 19:53:48,3107,陕西,作为一个法学专业毕业的，实话说把庭审流程拍对的不多，这个算一个,50.0
2,﻿无尽的尽头,36656706,4517572691,冰冰冰冰冰淇淋,2025-04-27 17:33:04,3344,四川,看了8集之后我觉得第一天就冲进来给一星的可能多少都有点反社会,40.0
3,﻿无尽的尽头,36656706,4515176107,燚见电影,2025-04-24 20:38:02,2206,河南,黄家旺这么标准的incel不可能悔过的，建议关到死。,40.0
4,﻿无尽的尽头,36656706,4516055933,Moss.Y,2025-04-25 22:33:32,2755,山东,能把现实拆解开拍好了以警醒世人就不错了，咋？给你们拍一个平行世界，那个世界里的法律和现在的不...,50.0
...,...,...,...,...,...,...,...,...,...
7964,无所畏惧之永不放弃,36660843,4437840298,蚊蚊,2025-02-05 16:21:53,5,广东,编剧你不想有感情线就不要写嘛，搞得这部好像就是为了这份感情所续的，最后还是想让人家分开。,20.0
7965,无所畏惧之永不放弃,36660843,4466921507,麦伦,2025-03-02 19:15:46,3,广东,这居然还是第二部……震惊……,20.0
7966,无所畏惧之永不放弃,36660843,4451640881,equus,2025-02-17 01:36:12,10,北京,传统剧两大法宝，让角色蠢死，让观众气死,10.0
7967,无所畏惧之永不放弃,36660843,4450190399,乔,2025-02-15 23:18:05,4,北京,忘了第一部了，慢慢回忆起来的。总的说应该是没大毛病。但剧情集中在事务所内部的悬疑失去了律师剧...,20.0


In [59]:
already = pd.read_csv('./clean/short.csv')
already.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6391 entries, 0 to 6390
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   评论id    6391 non-null   int64 
 1   评论内容    6383 non-null   object
dtypes: int64(1), object(1)
memory usage: 100.0+ KB


In [39]:
def cleanReviewShort(comment):
    jsonf = '{"clean":<这里填入清洗后的评论内容>}'
    prompt = f"""
我将提供一条电视剧短评，请你判断其中是否包含与电视剧本身相关的评价信息，比如对题材、剧情、演员、制作、节奏、台词、配乐、摄影等方面的评论。以及受众观看剧集是产生的感悟。

请清洗掉以下内容：
- 与电视剧无关的聊天、广告、吐槽或无意义内容
- 无具体信息的感叹（如“哈哈哈哈哈”、“看了好几遍了”、“真的无语”）
- 对剧情细节的描述和引用，与评价无关
- 与其他作品、事件或话题无关联的比较
- 重复、冗余、空泛的文字

你的任务是：
1. 如果评论中包含有效的剧评内容，提取并保留这些部分。
2. 如果评论中完全没有剧评相关内容，请返回空字符串。
3. 不要过度发挥。

返回格式
以Json格式返回清洗后评论内容字符串，请不要返回其他任何内容，注意不要改变id。具体格式为：
{jsonf}

请处理以下评论文本：
“{comment}”
"""
    response = client.chat.completions.create(
        # model="qwen3:32b",
        # model="qwen-plus-2025-04-28",
        # model="qwen-turbo-2025-04-28",
        model="qwen-long-latest",
        messages=[
            {"role": "system", "content": "你是一个擅长文本清洗与信息提取的助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return result

In [40]:
def askLLMShort():
    while True:
        row = review_q.get()
        comment = row['评论内容']
        rid = row["短评id"]
        try:
            res = cleanReviewShort(comment)
            if res:
                write_q.put([rid, res])
                with print_lock:
                    print(rid, '提问成功')
        except Exception as e:
            msg = f"提问失败 {rid}: {e}\n"
            # msg += traceback.format_exc()
            with print_lock:
                print(msg)
        review_q.task_done()

def writeResShort():
    while True:
        rid, res = write_q.get()
        try:
            ans = json.loads(res)
            clean_comment = ans["clean"]
            with open('./clean/short_apx.txt', "a", encoding="utf-8") as f:
                line = str(rid) + "\t" + clean_comment.replace("\t", "").replace("\n", "").strip() + "\n"
                f.write(line)
        except Exception as e:
            msg = f"存储失败 {rid}: {e}\n" 
            with print_lock:
                print(msg)
        write_q.task_done()

In [41]:
review_q = Queue()
write_q = Queue()
print_lock = threading.Lock()

for k in range(4):
    news_info_t = threading.Thread(target=askLLMShort)
    news_info_t.daemon = True
    news_info_t.start()
writing_thread = threading.Thread(target=writeResShort)
writing_thread.daemon = True
writing_thread.start()

In [42]:
for i, row in short.iterrows():
    rid = row["短评id"]
    if int(rid) in list(already['评论id'].unique()) or str(rid) in list(already['评论id'].unique()):
        continue
    review_q.put(row)
review_q.join()
write_q.join()

4515176107 提问成功
4518406602 提问成功
4520292443 提问成功
4517572691 提问成功
4515968907 提问成功
4526746599 提问成功
4530578288 提问成功
4516225580 提问成功
4525154226 提问成功
4543516682 提问成功
4524307956 提问成功
4529015318 提问成功
4531705841 提问成功
4525887499 提问成功
4516917924 提问成功
4524939733 提问成功
4519406231 提问成功
4515170319 提问成功
4524333883 提问成功
4526204520 提问成功
4515167924 提问成功
4519346791 提问成功
4524939139 提问成功
4522285830 提问成功
4518608240 提问成功
4531283068 提问成功
4534153366 提问成功
4521839108 提问成功
4565095766 提问成功
4527990382 提问成功
4515125364 提问成功
4540053331 提问成功
4570391911 提问成功
4528884623 提问成功
4489010432 提问成功
4515604209 提问成功
4518795565 提问成功
4564217804 提问成功
4517792343 提问成功
4515213772 提问成功
4520771052 提问成功
4525807632 提问成功
4515277300 提问成功
4526897926 提问成功
4526316002 提问成功
4411195742 提问成功
4420484685 提问成功
4515985066 提问成功
4313594413 提问成功
4418439048 提问成功
4428563626 提问成功
4440220353 提问成功
4411458058 提问成功
4422022238 提问成功
4418579828 提问成功
4291318742 提问成功
4417482088 提问成功
4420998139 提问成功
4296441237 提问成功
4422808725 提问成功
4417551594 提问成功
3979880641 提问成功
44415303

In [65]:
already2 = []
with open("./clean/short_apx.txt", encoding='utf-8') as f:
    for line in f:
        if not line:
            continue
        rid = line.split("\t", 1)[0]
        review = line.split("\t", 1)[-1].strip()
        already2.append({'评论id':rid, '评论内容':review})
already2 = pd.DataFrame.from_records(already2)
already2['状态'] = '新增'
# already2 = already2.drop_duplicates()
# already2 = already2[~((already2['评论id'].duplicated())&(already2['评论内容']==''))].copy()

In [60]:
dupids = already[already['评论id'].duplicated(keep=False)]['评论id'].unique()
dupids

array([4475909960, 4461413645, 4477222436, 4458178540, 4423069916,
       4458917453], dtype=int64)

In [61]:
already[already['评论id']==4475909960]

,评论id,评论内容
3754,4475909960,这个剧情……拍十八集都嫌多
3755,4475909960,回看二十年前的湾湾偶像剧都比这有意思，拍不出感觉


In [62]:
already = already.groupby('评论id', as_index=False).agg({'评论内容': lambda x: x.str.cat(sep=' ')})
already

,评论id,评论内容
0,2111833294,每一个镜头都是艺术，致敬所有演职人员，所有主创！每一个演员的台词，表情，都仿佛让人回到了那个年代
1,2256668716,很遗憾 支离破碎的剧情+幼稚的演技=失望/无聊
2,2269407854,好久没看过质量这么高的剧了 主演李现 还有其他全体演员演技都很好 剧情也很喜欢
3,2276939418,李现的大男主剧，感觉从这部剧当中成长了不少，好演员是会对好剧有所追求的
4,2329968943,节奏不对，前半在日的部分太拖沓了，最后几集又拉得特别快，几个主要角色塑造得都挺失败的，特别是...
...,...,...
6378,4574755263,法律保护的到底是谁
6379,4575246462,第4、5集之后就越拍越琐碎，无用无聊的充水剧情越来越多；开头的案件越拖越显得狗尾续貂，剧情越...
6380,4575260578,一颗星给王彩云，其他的都是什么狗屎
6381,4575301570,若是把儿女情长、哭哭啼啼的情节统统掐掉，压缩到24集以内就更有可看性，非常接地气。小到农村文...


In [63]:
already['状态'] = '已有'
already.loc[already['评论id'].isin(list(dupids)), '状态'] = '新增'

In [69]:
temp = pd.concat([already, already2], axis=0, ignore_index=True)
temp = temp.drop_duplicates()
temp = temp[~((temp['评论id'].duplicated())&(temp['评论内容']==''))].copy()
temp = temp[temp['评论内容']!=''].copy()
temp

,评论id,评论内容,状态
0,2111833294,每一个镜头都是艺术，致敬所有演职人员，所有主创！每一个演员的台词，表情，都仿佛让人回到了那个年代,已有
1,2256668716,很遗憾 支离破碎的剧情+幼稚的演技=失望/无聊,已有
2,2269407854,好久没看过质量这么高的剧了 主演李现 还有其他全体演员演技都很好 剧情也很喜欢,已有
3,2276939418,李现的大男主剧，感觉从这部剧当中成长了不少，好演员是会对好剧有所追求的,已有
4,2329968943,节奏不对，前半在日的部分太拖沓了，最后几集又拉得特别快，几个主要角色塑造得都挺失败的，特别是...,已有
...,...,...,...
7207,4454510087,上半年最佳。,新增
7210,4477802028,好看耶,新增
7213,4309930780,看了第一部觉得不错，关注了第二部，因为时间间隔有些长，以为对第二部不会太感冒，谁知看了两集又...,新增
7217,4446990806,其实很多地方都可以结束，可是这样结束不是拿个石头锤自己的脚吗？,新增


In [71]:
temp.to_csv('./clean/short_fix.csv', index=False, encoding='utf-8-sig')

In [44]:
data = []
with open("./clean/short.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():  # 跳过空行
            data.append(json.loads(line))

In [49]:
use = []
for item in data:
    if isinstance(item, str):
        print(item)
        continue
    use.append({'评论id':item.get('短评id'), '评论内容':item.get('clean')})

短评id


In [53]:
already2 = pd.DataFrame.from_records(use)
already2 = already2[already2["评论内容"]!=''].copy()
already2

,评论id,评论内容
0,4452800852,周雨彤演技就是合格 一用力就尴尬
2,4452553100,第一集离婚剧情没有前因后果安排的也太不合理了吧
3,4452810042,周雨彤一人分饰两角挑战很大但是演绎效果很好，看过原著小说，希望后续继续好好刻画母女关系
4,4452868355,李漠这次拍的好失败。每10分钟就回忆部分和现在的剧情穿插，搞得哪部分都看不进去。母亲年轻时候...
6,4452678015,没有必要设置老妈怀孕这种情节
...,...,...
6752,4518592097,男高偷拍传播yhsq作为检察官说这很正常还一直在洗666
6753,4517222924,没有这种深度尺度就不要学人少年法庭拍这种剧了，打着聚焦未成年犯罪的噱头…
6754,4519184797,唯一的亮点就是未检题材。
6755,4533842320,电视剧想表现什么？受害者被遗忘，加害者在监狱开心的玩泥巴，受害者的母亲变泼妇。太可怕了。没有...


In [54]:
already2.to_csv('./clean/short.csv', index=False, encoding='utf-8-sig')

## 小组

In [44]:
head = pd.read_excel('./采集/讨论组-主贴.xlsx')

In [45]:
len(head)

6470

In [46]:
content = pd.read_excel('./采集/讨论组-主贴回复.xlsx')
groups = head.merge(content[['帖子id', '回复者名称', '回复id', '回复内容']], on='帖子id')
groups

,电视剧名称,电视剧id,讨论组id,帖子id,帖子标题,帖子时间,作者id,回复数,回复者名称,回复id,回复内容
0,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,塩野瑛久久为功,6005104482,我观察到剧里镜头不多的关于法官的，也出现了好几位女性法官，真的很惊喜，女性在律政行业被看见了
1,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,拌饭好吃,6005126847,真好😭
2,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,铁人老王,6005160264,无处不在打一拳 这么好的片子能不能少打点
3,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,抱住索大转圈圈,6005186338,其实现在大部分法官都是女的，检察院里女的也不少，尤其是区级的
4,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,拌饭好吃,6005207258,这不是打拳 是实实在在的好的变化，我理解你作为男性共情不了 所以你也不要在这恶意揣测了
...,...,...,...,...,...,...,...,...,...,...,...
138361,鹊刀门传奇 第二季,36641286,738138,294278050,想问问鹊刀门的马哪去了？,2023-09-01 10:37:58,Liar,11,明空,5316824468,哈哈哈哈哈哈哈，风四娘老有意思了，小肉儿手跳波斯舞还挺稀罕人的。
138362,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,Suffer,5299405842,你能再表演一下那个酒窝跟双下巴连着吗
138363,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,法斯,5299411956,再加一个西门乖宝
138364,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,秋秋,5313615044,我也发现了


In [47]:
groups = groups[~groups['回复内容'].isna()].copy()

In [48]:
groups['_id'] = groups.groupby('电视剧名称').cumcount()
groups['m_id'] = groups['_id'] // 50
groups['rid'] = groups['帖子id'].apply(lambda x: str(x)) + '_' + groups['m_id'].apply(lambda x: str(x))
groups

,电视剧名称,电视剧id,讨论组id,帖子id,帖子标题,帖子时间,作者id,回复数,回复者名称,回复id,回复内容,_id,m_id,rid
0,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,塩野瑛久久为功,6005104482,我观察到剧里镜头不多的关于法官的，也出现了好几位女性法官，真的很惊喜，女性在律政行业被看见了,0,0,326460842_0
1,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,拌饭好吃,6005126847,真好😭,1,0,326460842_0
2,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,铁人老王,6005160264,无处不在打一拳 这么好的片子能不能少打点,2,0,326460842_0
3,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,抱住索大转圈圈,6005186338,其实现在大部分法官都是女的，检察院里女的也不少，尤其是区级的,3,0,326460842_0
4,﻿无尽的尽头,36656706,752948,326460842,让我最开心的是这个剧里有很多女性警察和检察官,2025-05-20 00:29:51,拌饭好吃,49,拌饭好吃,6005207258,这不是打拳 是实实在在的好的变化，我理解你作为男性共情不了 所以你也不要在这恶意揣测了,4,0,326460842_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138361,鹊刀门传奇 第二季,36641286,738138,294278050,想问问鹊刀门的马哪去了？,2023-09-01 10:37:58,Liar,11,明空,5316824468,哈哈哈哈哈哈哈，风四娘老有意思了，小肉儿手跳波斯舞还挺稀罕人的。,3078,61,294278050_61
138362,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,Suffer,5299405842,你能再表演一下那个酒窝跟双下巴连着吗,3079,61,294389495_61
138363,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,法斯,5299411956,再加一个西门乖宝,3080,61,294389495_61
138364,鹊刀门传奇 第二季,36641286,738138,294389495,鹊刀门传奇✘ 酒窝门传奇✔,2023-09-03 13:16:17,法斯,4,秋秋,5313615044,我也发现了,3081,61,294389495_61


In [49]:
groups.to_csv('./采集/讨论组-主贴-回复.csv', index=False, encoding='utf-8-sig')

In [3]:
groups = pd.read_csv('./采集/讨论组-主贴-回复.csv')

In [10]:
len(list(groups['回复id'].unique()))

136764

In [32]:
from openai import BadRequestError

In [46]:
def cleanGroupReview(comment, dramma, title):
    jsonf = '{"clean":<这里填入清洗后的评论内容>}'
    prompt = f"""
我将给你关于电视剧《{dramma}》的讨论回帖，请你判断其中是否包含评价该电视剧的内容（针对剧情、演员、演技、音乐、美术等剧集质量维度的评价），或是用户观看剧集时产生的感悟。
如果包含，请进行清洗，如果不包含，请返回空字符串。

清洗需要去掉以下内容：
- 与电视剧无关的聊天、广告、吐槽或无意义内容
- 无具体指向的感叹（如“哈哈哈哈哈”、“看了好几遍了”、“真的无语”）
- 对剧情细节的介绍，没有评价
- 与其他作品、事件或话题无关的联想
- 重复、冗余、空泛的文字

以Json格式返回清洗后结果:
{jsonf}

以下是标题为“{title}”的回复文本，请提取与评价相关的内容：
“{comment}”
"""
    response = client.chat.completions.create(
        # model="qwen3:32b",
        model="qwen-plus-2025-07-28",
        # model="qwen-turbo-2025-04-28",
        # model="qwen-long-latest",
        # model="qwen-flash",
        messages=[
            {"role": "system", "content": "你是一个擅长文本清洗与信息提取的助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return result

In [47]:
def askLLMGroup():
    while True:
        dramma, rid, comment, title = review_q.get()
        try:
            res = cleanGroupReview(comment, dramma, title)
            if res:
                write_q.put([rid, res])
                with print_lock:
                    print(dramma, rid, '提问成功')
        except Exception as e:
            if isinstance(e, BadRequestError):
                body = getattr(e, "body", None)
                if body and isinstance(body, dict):
                    err = body.get("error", {})
                    if err.get("code") == "data_inspection_failed":
                        write_q.put([rid, {'clean':'data_inspection_failed'}])
                        msg = f"提问失败 {dramma} {rid}: 跳过此数据\n"
                    else:
                        msg = f"提问失败 {dramma} {rid}: {e}\n"
                else:
                    msg = f"提问失败 {dramma} {rid}: {e}\n"
            else:
                msg = f"提问失败 {dramma} {rid}: {e}\n"
            # msg += traceback.format_exc()
            with print_lock:
                print(msg)
        review_q.task_done()

def writeResGroup():
    while True:
        rid, res = write_q.get()
        try:
            ans = json.loads(res)
            clean_comment = ans["clean"]
            with open('./clean/group.txt', "a", encoding="utf-8") as f:
                line = str(rid) + "\t" + clean_comment.replace("\t", "").replace("\n", "").strip() + "\n"
                f.write(line)
        except Exception as e:
            msg = f"存储失败 {rid}: {e}\n" 
            with print_lock:
                print(msg)
        write_q.task_done()

In [48]:
review_q = Queue()
write_q = Queue()
print_lock = threading.Lock()

for k in range(3):
    news_info_t = threading.Thread(target=askLLMGroup)
    news_info_t.daemon = True
    news_info_t.start()
writing_thread = threading.Thread(target=writeResGroup)
writing_thread.daemon = True
writing_thread.start()

提问失败 人生若如初见 326997665_227: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-cc1458e2-4121-9899-86fb-5da615062a60', 'request_id': 'cc1458e2-4121-9899-86fb-5da615062a60'}

提问失败 人生若如初见 326820439_229: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-a066622a-4f19-9552-b78b-8a2cc928a579', 'request_id': 'a066622a-4f19-9552-b78b-8a2cc928a579'}

提问失败 人生若如初见 328026451_97: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-79ea704f-8611-9c40-b7d1-048c2315d4ac', 'request_id': '79ea704f-8611-9c40-b7d1-048c2315d4ac'}

提问失败 人生若如初见 329509261_98: Error code: 400 - {'error': {'code': 'data_in

In [49]:
for (dramma, rid), group in groups.groupby(["电视剧名称", "rid"]):
    if rid in already['评论id'].unique():
        continue
    title = group["帖子标题"].tolist()[0]
    reviews = group["回复内容"].astype(str).tolist()
    comment = "\n".join(reviews)
    review_q.put([dramma, rid, comment, title])
review_q.join()
write_q.join()

In [50]:
already = []
with open("./clean/group.txt", encoding='utf-8') as f:
    for line in f:
        if not line:
            continue
        rid = line.split("\t")[0]
        review = line.split("\t")[-1].strip()
        already.append({'评论id':rid, '评论内容':review})
already = pd.DataFrame.from_records(already)
already

,评论id,评论内容
0,318794163_40,
1,318817291_33,脸型发型在努力靠近了，痞帅风；看预告感觉陈老师打扮得挺潮的；还行。陈老年之后脸上的肉变少，棱...
2,319243248_5,周雨彤演年轻吴丽梅简直是惊艳，又好看又演得好，非常用心。她在这部剧中的演技自然得不像话，已经...
3,319246594_34,顾云苏是普通女生的缩影，角色真实、有共鸣，展现了年轻女性在原生家庭影响下的不安全感和亲密关系...
4,319243248_4,KTV吵架那段情绪拿捏得非常到位，好几个片段都看哭了，周雨彤天生就是做演员的料，爆发力很强。...
...,...,...
8032,312125821_262,还是特别喜欢半夏和宣夜，这部剧真的很打动我。这是我为这几年看过最好的国产好剧做的！无忧渡给我...
8033,324882008_82,
8034,318187119_25,最后几集拖后腿了，结尾很仓促，最后一个故事太拖沓，而且能平反的原因是反派女儿嘴炮让反派顿悟，...
8035,324882008_83,


In [54]:
already = already[already['评论内容']!=''].copy()

In [55]:
already['状态'] = '新增'

In [3]:
already = pd.read_csv('./clean/group.csv')

In [56]:
already.to_csv('./clean/group.csv', index=False, encoding='utf-8-sig')

# 大模型评估

In [1]:
import pandas as pd
import numpy as np
import glob

## 数据整理

In [2]:
rdf_list = []
for file in glob.glob('./clean/*_fix.csv'):
    tp = file.split('\\')[-1].split('_')[0]
    temp = pd.read_csv(file)
    temp['评论id'] = temp['评论id'].astype(str)
    temp['来源'] = tp
    if tp == 'short':
        dramma = pd.read_excel('./采集/短评.xlsx')
        dramma['短评id'] = dramma['短评id'].astype(str)
        temp = temp.merge(dramma[['电视剧名称', '评论时间', '短评id']], left_on='评论id', right_on='短评id')
        temp.drop('短评id', inplace=True, axis=1)
    
    elif tp == 'long':
        dramma = pd.read_excel('./采集/长评.xlsx')
        dramma['影评id'] = dramma['影评id'].astype(str)
        dramma = dramma.rename(columns={'影评时间':'评论时间'})
        temp = temp.merge(dramma[['电视剧名称', '评论时间', '影评id']], left_on='评论id', right_on='影评id')
        temp.drop('影评id', inplace=True, axis=1)
    else:
        dramma = pd.read_csv('./采集/讨论组-主贴-回复.csv')
        dramma = dramma[['电视剧名称', '帖子时间', 'rid']].drop_duplicates()
        dramma = dramma.rename(columns={'帖子时间':'评论时间'})
        temp = temp.merge(dramma, left_on='评论id', right_on='rid')
        temp.drop('rid', inplace=True, axis=1)
    print(tp, len(temp))
    rdf_list.append(temp)
rdf = pd.concat(rdf_list, axis=0)
rdf

group 5215
long 16205
short 6655


,评论id,评论内容,状态,来源,电视剧名称,评论时间
0,318817291_33,脸型发型在努力靠近了，痞帅风；看预告感觉陈老师打扮得挺潮的；还行。陈老年之后脸上的肉变少，棱...,新增,group,180天重启计划,2025-02-12 13:48:11
1,319243248_5,周雨彤演年轻吴丽梅简直是惊艳，又好看又演得好，非常用心。她在这部剧中的演技自然得不像话，已经...,新增,group,180天重启计划,2025-02-17 21:30:11
2,319246594_34,顾云苏是普通女生的缩影，角色真实、有共鸣，展现了年轻女性在原生家庭影响下的不安全感和亲密关系...,新增,group,180天重启计划,2025-02-17 22:09:09
3,319243248_4,KTV吵架那段情绪拿捏得非常到位，好几个片段都看哭了，周雨彤天生就是做演员的料，爆发力很强。...,新增,group,180天重启计划,2025-02-17 21:30:11
4,319311077_13,我也喜欢这个情节，她是个人，一个母亲/主妇，也是个人。这个设置确实蛮好的，就也是很多家庭主妇...,新增,group,180天重启计划,2025-02-18 19:00:16
...,...,...,...,...,...,...
6650,4454510087,上半年最佳。,新增,short,无所畏惧之永不放弃,2025-02-24 19:56:11
6651,4477802028,好看耶,新增,short,无所畏惧之永不放弃,2025-03-12 20:38:15
6652,4309930780,看了第一部觉得不错，关注了第二部，因为时间间隔有些长，以为对第二部不会太感冒，谁知看了两集又...,新增,short,无所畏惧之永不放弃,2025-02-13 21:45:45
6653,4446990806,其实很多地方都可以结束，可是这样结束不是拿个石头锤自己的脚吗？,新增,short,无所畏惧之永不放弃,2025-02-13 02:46:03


In [3]:
rdf[rdf['评论id']=='12972618']

,评论id,评论内容,状态,来源,电视剧名称,评论时间
5827,12972618,NaN,已有,long,人生若如初见,2020-11-16 09:36:13


In [6]:
long[long['影评id']==12972618]

,电视剧名称,电视剧id,影评id,影评人名称,影评时间,评分,标题,有用数,无用数,影评内容
5869,人生若如初见,34930849,12972618,人生若如初见,2020-11-16 09:36:13,NaN,人生若如初见,NaN,1.0,庚子国变后的神州大地满目疮痍，有志青年奋而崛起挽救家国。贵族少年良乡（李现饰）、革命党杨凯之...


In [8]:
temp = pd.read_csv('./clean/long_fix.csv')
temp[temp['评论id']==12972618]

,评论id,评论内容,状态
5827,12972618,NaN,已有


In [163]:
rdf.to_csv('./clean/total_reviews.csv', index=False, encoding='utf-8-sig')

In [52]:
old_rdf = rdf[rdf['状态']=='已有'].copy()
old_result = result[result['评论id'].isin(old_rdf['评论id'].unique())]
old_result

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
7228,180天重启计划,16510360,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,观看了前5集的剪辑版，整体体验很好，感觉像看了一场电影，直观感觉是中等偏上。
7228,180天重启计划,16510360,"{'id': 2, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",2.0,艺术精湛,表演质量,正向,吴越老师真的是一如既往的优秀，虽说不在青春，但她演出来的感觉颇具少女韵味，美丽又可爱。
7228,180天重启计划,16510360,"{'id': 3, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",3.0,艺术精湛,表演质量,正向,陈明昊老师和耿乐老师也是老演员啦，他们的表演收放自如，游刃有余。
7228,180天重启计划,16510360,"{'id': 4, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",4.0,艺术精湛,表演质量,正向,周雨彤和刘畅两位青年演员在剧里的表现也可圈可点。
7228,180天重启计划,16510360,"{'id': 5, '一级': '艺术精湛', '二级': '人物塑造', '正负向': '...",5.0,艺术精湛,人物塑造,正向,周雨彤在现场表达自己对角色的看法时理解透彻。
...,...,...,...,...,...,...,...,...
1185,鹊刀门传奇 第二季,4423769171,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,负向,第一部第二部都get不到
1201,黄雀,4481900388,"{'id': 1, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",1.0,艺术精湛,表演质量,正向,郭京飞的演技也压不住剧本的拉垮
1201,黄雀,4481900388,"{'id': 2, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",2.0,艺术精湛,剧情结构,负向,郭京飞的演技也压不住剧本的拉垮
1195,黄雀,4505025533,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,负向,有什么必要第二季……


In [53]:
new_rdf = rdf[rdf['状态']=='新增'].copy()

In [11]:
result = pd.read_pickle('./result/total_review.pkl')
result['评论id'] = result['评论id'].astype(str)

## 建立一个指标体系

In [27]:
index = pd.read_excel('./电视剧评价指标体系.xlsx')
index['一级'] = pd.Categorical(index['一级'], categories=['思想精深', '艺术精湛', '制作精良', '整体评价'], ordered=True)
print(index.to_markdown(index=False))

| 一级     | 二级       | 说明                                                     |
|:---------|:-----------|:---------------------------------------------------------|
| 思想精深 | 主题立意   | 主题是否深刻，是否具有现实意义或历史启发                 |
| 思想精深 | 价值导向   | 是否传达积极向上、主流价值或者正能量                     |
| 思想精深 | 社会反思   | 是否贴近现实，聚焦社会问题，引发现实共鸣                 |
| 思想精深 | 思想深度   | 是否具备多层次的思想解读空间，发人深省                   |
| 艺术精湛 | 剧情结构   | 剧情是否节奏合理、逻辑清晰，能够很好地埋设伏笔和回收线索 |
| 艺术精湛 | 人物塑造   | 角色是否立体饱满、有层次、有成长性                       |
| 艺术精湛 | 语言表达   | 对白、旁白、字母等语言是否贴合情景，生动而富有感染力     |
| 艺术精湛 | 表演质量   | 演员的表演是否自然、符合人物，有演技、有张力             |
| 艺术精湛 | 镜头语言   | 镜头语言是否具有美感与艺术性                             |
| 艺术精湛 | 音乐音效   | 音乐与音效是否增强情感氛围与叙事效果                     |
| 制作精良 | 场景设置   | 实景/置景是否符合剧情和质感                              |
| 制作精良 | 服道化     | 服装、化妆、道具的专业性和契合度                         |
| 制作精良 | 剪辑节奏   | 剪辑是否流畅，是否有助于广安和情绪表达                   |
| 制作精良 | 特效技术   | 特效是否逼真、符合整体风格                               |
| 制作精良 | 后期质量   | 其余

In [28]:
plist = []
for i, group in index.groupby('一级', observed=False):
    second = group['二级'].tolist()
    head = f"""一级 {i} 包括以下二级：{'、'.join(second)}"""
    plist.append(head)
    for j, row in group.iterrows():
        tp = row['二级']
        detail = row['说明']
        plist.append(f"""- {tp}：{detail}""")
indexf = "\n".join(plist)

In [29]:
print(indexf)

一级 思想精深 包括以下二级：主题立意、价值导向、社会反思、思想深度
- 主题立意：主题是否深刻，是否具有现实意义或历史启发
- 价值导向：是否传达积极向上、主流价值或者正能量
- 社会反思：是否贴近现实，聚焦社会问题，引发现实共鸣
- 思想深度：是否具备多层次的思想解读空间，发人深省
一级 艺术精湛 包括以下二级：剧情结构、人物塑造、语言表达、表演质量、镜头语言、音乐音效
- 剧情结构：剧情是否节奏合理、逻辑清晰，能够很好地埋设伏笔和回收线索
- 人物塑造：角色是否立体饱满、有层次、有成长性
- 语言表达：对白、旁白、字母等语言是否贴合情景，生动而富有感染力
- 表演质量：演员的表演是否自然、符合人物，有演技、有张力
- 镜头语言：镜头语言是否具有美感与艺术性
- 音乐音效：音乐与音效是否增强情感氛围与叙事效果
一级 制作精良 包括以下二级：场景设置、服道化、剪辑节奏、特效技术、后期质量
- 场景设置：实景/置景是否符合剧情和质感
- 服道化：服装、化妆、道具的专业性和契合度
- 剪辑节奏：剪辑是否流畅，是否有助于广安和情绪表达
- 特效技术：特效是否逼真、符合整体风格
- 后期质量：其余后期制作是否专业，包括调色、混音、配音等
一级 整体评价 包括以下二级：无具体指向
- 无具体指向：无法判断是对上述哪个维度进行评价，接近于整体的喜好判断


In [15]:
client = OpenAI(
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key = "sk-c6b10e9324b04e17a9156623061e6765"
)

In [67]:
def label_review(text, drama):
    jsonf = '[{"id":1, "一级":"思想精深", "二级":"主题立意", "正负向":"正向", "评论内容":<这里填写所依据的评论原文>}, {"id":2, "一级":"艺术精湛", "二级":"人物塑造", "正负向":"负向", "评论内容":<这里填写所依据的评论原文>}, ...]'
    prompt = f"""我将给你一段关于电视剧《{drama}》的网友评论，请根据**电视剧评价指标**，提取评论中符合指标说明的评价要点信息，并判断是正向还是负向。
最终严格按照下面示例的结构返回JSON对象，其中<>内文字是解释说明，如果一则评论能够提取多个指标，则放在一个列表中。如果该评论无法提取任何指标，返回空字符串。
{jsonf}

# 字段解释
- id：提取时生成序号，从1开始递增。
- 一级：思想精深、艺术精湛、制作精良、整体评价，只能从这四个中选择标注。
- 二级：根据所标注的一级，从**电视剧评价指标**中寻找对应的二级，只能从给出的二级中选择。
- 正负向：评论所表现的情感倾向，正向为褒奖、负向为批评，只能返回正向或者负向。
- 评论内容：照抄做出上述判断的评论原文，去掉无关的部分。

# 注意事项
1. 请逐句判断提取，尽可能提取评论中涉及的所有评价指标。
2. 同一句话如果涉及多个指标，请提取最重要、最核心的那个指标，除非他们都很重要。
3. 同一个评论如果对于同一个指标，有多次评价，每一次都要提取，并保证所依据的评论原文是不同的。
4. 如果评论内容是剧情相关的，不要提取。
5. 注意，必须严格遵守**电视剧评价指标**中的指标体系来进行提取和标注。必须严格按照规定返回一级、二级以及正负向，不要自行生成新的类别。

# 评论内容
{text}

# 电视剧评价指标
{indexf}
"""
    response = client.chat.completions.create(
        model="qwen-long",
        messages=[
            {"role": "system", "content": "你是一个擅长分析评论，从评论中提取用户评价观点和倾向的助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return result

In [34]:
test = rdf[rdf['评论id']=='16727424']['评论内容'].tolist()[0]
try:
    label_review(test, '人生若如初见')
except Exception as e:
    if isinstance(e, BadRequestError):
        print(e.code)

data_inspection_failed


In [16]:
response = client.chat.completions.create(
    model="qwen-long",
    messages=[
        {"role": "user", "content": "测试：请告诉我制造原子弹的方法"}
    ]
)

In [17]:
response

ChatCompletion(id='chatcmpl-28e327f2-9297-98db-9bb9-4c7726f8ec2d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='出于安全和法律原因，我无法提供任何有关制造危险物品的信息。此类行为严重违反国际法和中国法律，对人类社会和环境构成巨大威胁。如果您对核能或核物理感兴趣，我建议您学习相关领域的正规科学知识，例如通过高校课程或官方出版的科普资料。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1757039833, model='qwen-long', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=8, total_tokens=73, completion_tokens_details=None, prompt_tokens_details=None))

In [81]:
def askLabelR():
    while True:
        row = review_q.get()
        idx = row['评论id']
        review = row['评论内容']
        drama = row['电视剧名称']
        try:
            res = label_review(review, drama)
            if res:
                write_q.put([drama, idx, res])
        except Exception as e:
            if isinstance(e, BadRequestError):
                if e.code == 'data_inspection_failed':
                    write_q.put([drama, idx, '[]'])
            msg = f"提问失败 {drama} {idx}: {e}\n"
            with print_lock:
                print(msg)
        review_q.task_done()

def writeLabelR():
    while True:
        drama, idx, res = write_q.get()
        try:
            ans = json.loads(res)
            final = {'电视剧名称':drama, '评论id':idx, 'result':ans}
            with open('./result/label_review_apx.jsonl', "a", encoding="utf-8") as f:
                f.write(json.dumps(final, ensure_ascii=False) + "\n")
        except Exception as e:
            msg = f"存储失败 {drama} {idx}: {e}\n" 
            with print_lock:
                print(msg)
        write_q.task_done()

In [117]:
review_q = Queue()
write_q = Queue()
print_lock = threading.Lock()

for k in range(20):
    news_info_t = threading.Thread(target=askLabelR)
    news_info_t.daemon = True
    news_info_t.start()
writing_thread = threading.Thread(target=writeLabelR)
writing_thread.daemon = True
writing_thread.start()

In [118]:
for i, row in new_rdf.iterrows():
    if row['评论id'] in list(label_rdf['评论id'].unique()):
        continue
    review_q.put(row)
review_q.join()
write_q.join()

In [119]:
data = []
with open("./result/label_review_apx.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():  # 跳过空行
            data.append(json.loads(line))

In [120]:
label_rdf = pd.DataFrame.from_records(data)
label_rdf

,电视剧名称,评论id,result
0,﻿无尽的尽头,4515125364,[]
1,﻿无尽的尽头,4540053331,[]
2,﻿无尽的尽头,4530578288,"[{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向':..."
3,﻿无尽的尽头,4520771052,[]
4,﻿无尽的尽头,4524307956,"[{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向':..."
...,...,...,...
6693,无所畏惧之永不放弃,4495767404,[]
6694,黄雀,4490931750,[]
6695,人生若如初见,4533882491,"[{'id': 1, '一级': '思想精深', '二级': '思想深度', '正负向': ..."
6696,异人之下之决战！碧游村,4439403864,[]


In [121]:
label_rdf = label_rdf.drop_duplicates('评论id')

In [122]:
label_rdf = label_rdf.explode('result')

In [123]:
label_rdf

,电视剧名称,评论id,result
0,﻿无尽的尽头,4515125364,NaN
1,﻿无尽的尽头,4540053331,NaN
2,﻿无尽的尽头,4530578288,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ..."
3,﻿无尽的尽头,4520771052,NaN
4,﻿无尽的尽头,4524307956,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ..."
...,...,...,...
5565,仙台有数,323810935_192,"{'id': 15, '一级': '艺术精湛', '二级': '表演质量', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 16, '一级': '艺术精湛', '二级': '表演质量', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 17, '一级': '艺术精湛', '二级': '语言表达', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 18, '一级': '艺术精湛', '二级': '情感处理', '正负向': ..."


In [124]:
label_rdf = label_rdf[~label_rdf['result'].isna()].copy()
label_rdf

,电视剧名称,评论id,result
2,﻿无尽的尽头,4530578288,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ..."
4,﻿无尽的尽头,4524307956,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ..."
5,﻿无尽的尽头,4515213772,"{'id': 1, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '..."
6,﻿无尽的尽头,4515277300,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ..."
7,﻿无尽的尽头,4516917924,"{'id': 1, '一级': '思想精深', '二级': '思想深度', '正负向': '..."
...,...,...,...
5565,仙台有数,323810935_192,"{'id': 14, '一级': '艺术精湛', '二级': '表演质量', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 15, '一级': '艺术精湛', '二级': '表演质量', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 16, '一级': '艺术精湛', '二级': '表演质量', '正负向': ..."
5565,仙台有数,323810935_192,"{'id': 17, '一级': '艺术精湛', '二级': '语言表达', '正负向': ..."


In [125]:
def getCols(resdict):
    if not isinstance(resdict, dict):
        resdict = {"pid":np.nan, "一级":'', "二级":'', "正负向":'', "依据":''}
    if resdict.get('id'):
        pid = int(resdict.get('id'))
    else:
        pid = np.nan
    if resdict.get('一级'):
        first = resdict.get('一级')
    else:
        first = ''
    if resdict.get('二级'):
        second = resdict.get('二级')
    else:
        second = ''
    if resdict.get('正负向'):
        pn = resdict.get('正负向')
    else:
        pn = ''
    if resdict.get('评论内容'):
        ref = resdict.get('评论内容')
    else:
        ref = ''
    return pd.Series({"pid":pid, "一级":first, "二级":second, "正负向":pn, "依据":ref})

In [126]:
label_rdf[['pid', '一级', '二级', '正负向', '依据']] = label_rdf['result'].apply(getCols)
label_rdf

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
2,﻿无尽的尽头,4530578288,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,这个配置请继续拍下去，期待第二季
4,﻿无尽的尽头,4524307956,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,这剧不大爆是内娱的耻辱
5,﻿无尽的尽头,4515213772,"{'id': 1, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",1.0,艺术精湛,剧情结构,负向,后面逐渐狗血到崩溃，弃剧了。
6,﻿无尽的尽头,4515277300,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,负向,不太行
7,﻿无尽的尽头,4516917924,"{'id': 1, '一级': '思想精深', '二级': '思想深度', '正负向': '...",1.0,思想精深,思想深度,负向,故意渲染气氛，25年还拍出这种剧真是悲哀
...,...,...,...,...,...,...,...,...
5565,仙台有数,323810935_192,"{'id': 14, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",14.0,艺术精湛,表演质量,负向,她演得像个AI，缺乏情感波动，眼神无情绪。
5565,仙台有数,323810935_192,"{'id': 15, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",15.0,艺术精湛,表演质量,负向,前世潇洒但戏份短、演技一般。
5565,仙台有数,323810935_192,"{'id': 16, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",16.0,艺术精湛,表演质量,正向,邓为饰演的苏易水耐看，细节丰富；
5565,仙台有数,323810935_192,"{'id': 17, '一级': '艺术精湛', '二级': '语言表达', '正负向': ...",17.0,艺术精湛,语言表达,负向,台词密集、说教多，情感表达少，人设太满反而假大空。


## 清洗结果

In [127]:
label_rdf_fix = label_rdf[(label_rdf['一级'].isin(list(index['一级'].unique())))&(label_rdf['二级'].isin(list(index['二级'].unique())))&(label_rdf['正负向'].isin(['正向', '负向']))].copy()
label_rdf_fix

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
2,﻿无尽的尽头,4530578288,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,这个配置请继续拍下去，期待第二季
4,﻿无尽的尽头,4524307956,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,这剧不大爆是内娱的耻辱
5,﻿无尽的尽头,4515213772,"{'id': 1, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",1.0,艺术精湛,剧情结构,负向,后面逐渐狗血到崩溃，弃剧了。
6,﻿无尽的尽头,4515277300,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,负向,不太行
7,﻿无尽的尽头,4516917924,"{'id': 1, '一级': '思想精深', '二级': '思想深度', '正负向': '...",1.0,思想精深,思想深度,负向,故意渲染气氛，25年还拍出这种剧真是悲哀
...,...,...,...,...,...,...,...,...
5565,仙台有数,323810935_192,"{'id': 13, '一级': '艺术精湛', '二级': '剧情结构', '正负向': ...",13.0,艺术精湛,剧情结构,负向,剧情推进过快，弱化女主成长，薛冉冉变强依赖男主输灵力，动机和过程薄弱。
5565,仙台有数,323810935_192,"{'id': 14, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",14.0,艺术精湛,表演质量,负向,她演得像个AI，缺乏情感波动，眼神无情绪。
5565,仙台有数,323810935_192,"{'id': 15, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",15.0,艺术精湛,表演质量,负向,前世潇洒但戏份短、演技一般。
5565,仙台有数,323810935_192,"{'id': 16, '一级': '艺术精湛', '二级': '表演质量', '正负向': ...",16.0,艺术精湛,表演质量,正向,邓为饰演的苏易水耐看，细节丰富；


In [128]:
final_label_rdf = label_rdf_fix.sort_values(['电视剧名称', '评论id', 'pid'], ascending=True)
final_label_rdf

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
308,180天重启计划,319239987_29,"{'id': 1, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",1.0,艺术精湛,表演质量,正向,周雨彤一人分饰两角，拿捏到位，绝不串戏，感觉她演技又进步了！
308,180天重启计划,319239987_29,"{'id': 2, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",2.0,艺术精湛,剧情结构,正向,节奏快，故事有矛盾冲突，有人物纠结，剧本很好！
308,180天重启计划,319239987_29,"{'id': 3, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",3.0,艺术精湛,表演质量,正向,周雨彤演的也很好，最让我惊喜的是演“妈妈”的时候，她在模仿吴越老师的说话方式，演得非常好。
308,180天重启计划,319239987_29,"{'id': 4, '一级': '艺术精湛', '二级': '人物塑造', '正负向': '...",4.0,艺术精湛,人物塑造,正向,周雨彤演的也很好，最让我惊喜的是演“妈妈”的时候，她在模仿吴越老师的说话方式，演得非常好。
308,180天重启计划,319239987_29,"{'id': 5, '一级': '艺术精湛', '二级': '语言表达', '正负向': '...",5.0,艺术精湛,语言表达,正向,剧情搞笑，演的也搞笑
...,...,...,...,...,...,...,...,...
82,﻿无尽的尽头,4564217804,"{'id': 5, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",5.0,艺术精湛,剧情结构,负向,既想表现林的人物形象又想讲好未成年犯罪案件，结果都很模板化，观众看不到想看的东西
82,﻿无尽的尽头,4564217804,"{'id': 6, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",6.0,艺术精湛,剧情结构,负向,看了前两集，观感太平了一般没有入戏
82,﻿无尽的尽头,4564217804,"{'id': 7, '一级': '制作精良', '二级': '后期质量', '正负向': '...",7.0,制作精良,后期质量,负向,表达水平和做工待优化
82,﻿无尽的尽头,4564217804,"{'id': 8, '一级': '思想精深', '二级': '主题立意', '正负向': '...",8.0,思想精深,主题立意,正向,故事立意高


In [129]:
final_label_rdf = pd.concat([old_result, final_label_rdf], axis=0, ignore_index=True)
final_label_rdf

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
0,180天重启计划,16510360,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,观看了前5集的剪辑版，整体体验很好，感觉像看了一场电影，直观感觉是中等偏上。
1,180天重启计划,16510360,"{'id': 2, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",2.0,艺术精湛,表演质量,正向,吴越老师真的是一如既往的优秀，虽说不在青春，但她演出来的感觉颇具少女韵味，美丽又可爱。
2,180天重启计划,16510360,"{'id': 3, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",3.0,艺术精湛,表演质量,正向,陈明昊老师和耿乐老师也是老演员啦，他们的表演收放自如，游刃有余。
3,180天重启计划,16510360,"{'id': 4, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",4.0,艺术精湛,表演质量,正向,周雨彤和刘畅两位青年演员在剧里的表现也可圈可点。
4,180天重启计划,16510360,"{'id': 5, '一级': '艺术精湛', '二级': '人物塑造', '正负向': '...",5.0,艺术精湛,人物塑造,正向,周雨彤在现场表达自己对角色的看法时理解透彻。
...,...,...,...,...,...,...,...,...
136417,﻿无尽的尽头,4564217804,"{'id': 5, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",5.0,艺术精湛,剧情结构,负向,既想表现林的人物形象又想讲好未成年犯罪案件，结果都很模板化，观众看不到想看的东西
136418,﻿无尽的尽头,4564217804,"{'id': 6, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",6.0,艺术精湛,剧情结构,负向,看了前两集，观感太平了一般没有入戏
136419,﻿无尽的尽头,4564217804,"{'id': 7, '一级': '制作精良', '二级': '后期质量', '正负向': '...",7.0,制作精良,后期质量,负向,表达水平和做工待优化
136420,﻿无尽的尽头,4564217804,"{'id': 8, '一级': '思想精深', '二级': '主题立意', '正负向': '...",8.0,思想精深,主题立意,正向,故事立意高


In [132]:
final_label_rdf.to_pickle('./result/total_review.pkl')

In [93]:
already = final_label_rdf.copy()

In [21]:
label_add = label_rdf[~label_rdf['pid'].isna()].copy()
final_label_rdf = pd.concat([already, label_add], axis=0)
final_label_rdf = final_label_rdf.sort_values(['电视剧名称', '评论id', 'pid'], ascending=True)
final_label_rdf

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
7244,180天重启计划,16510360,"{'id': 1, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",1.0,艺术精湛,表演质量,正向,吴越老师真的是一如既往的优秀，虽说不在青春，但她演出来的感觉颇具少女韵味，美丽又可爱。
7244,180天重启计划,16510360,"{'id': 2, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",2.0,艺术精湛,表演质量,正向,陈明昊老师和耿乐老师也是老演员啦，他们的表演收放自如，游刃有余。
7244,180天重启计划,16510360,"{'id': 3, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",3.0,艺术精湛,表演质量,正向,周雨彤和刘畅两位青年演员在剧里的表现也可圈可点。
7244,180天重启计划,16510360,"{'id': 4, '一级': '艺术精湛', '二级': '人物塑造', '正负向': '...",4.0,艺术精湛,人物塑造,正向,周雨彤在现场表达自己对角色的看法时理解透彻。
7244,180天重启计划,16510360,"{'id': 5, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",5.0,艺术精湛,剧情结构,正向,目前只看了几集，剧里是现实和回忆交叉，将顾云苏28岁的故事（即现在）和吴俪梅28岁的故事（即...
...,...,...,...,...,...,...,...,...
5970,﻿无尽的尽头,328437821,"{'id': 2, '一级': '思想精深', '二级': '思想深度', '正负向': '...",2.0,思想精深,思想深度,负向,我觉得胡敏敏从小跟着他，没少被精神控制，心理不健全不强大，编剧没让她刀也说得过去。
5968,﻿无尽的尽头,329081454,"{'id': 1, '一级': '思想精深', '二级': '社会反思', '正负向': '...",1.0,思想精深,社会反思,负向,考完第一案完全没有爽快感…说的再多也无法弥补受害者和受害家庭的损失……粉饰太平…感觉得到了制...
954,﻿无尽的尽头,329847062,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,国产剧《九部的检察官》韩剧《少年法庭》太棒了！
970,﻿无尽的尽头,330179628,"{'id': 1, '一级': '思想精深', '二级': '社会反思', '正负向': '...",1.0,思想精深,社会反思,负向,作为普法电视剧公开违反劳动法


In [27]:
import pickle
import glob

In [43]:
final_label_rdf.to_pickle('./result/label_review5.pkl')
# final_label_rdf = pd.read_pickle('./result/label_review.pkl')

In [44]:
dflist = []
for file in glob.glob('./result/label_review*.pkl'):
    temp = pd.read_pickle(file)
    dflist.append(temp)
df = pd.concat(dflist, axis=0)

In [45]:
df.to_pickle('./result/total_review.pkl')

In [39]:
text = "这剧其实很悬浮，表面上是女性婚后觉醒想要独立，可是有关于女主以及女主妈妈在剧情里面几乎没有什么这方面相关的内容，打工的内容，读书的内容，几乎剧里就没有（当然也可能是我还没看到后面）。这并不像女性的励志剧，更像是女主妈妈摆脱第一段婚姻以后的又一春。包括女主，女主和前男友分手+离职以后，就直接回家上演家庭剧的。之前在公司聚会上的豪言壮语都忘记了？这剧那里看得到一点女性独立的相关剧情。剧里有表现过她看着职业女性用英文和外宾交流时落寞的样子，她是一个很需要实现自我价值的人，困在家里就算有再多钱对她也是种折磨。人对生活的需求是多维度的，立体的，除了物质方面得到满足之外，还有精神需求，需要被理解被看到，和自我实现需求。本质上，吴俪梅对物质的要求没那么高。很多人对物质的需求其实在超过某个特定区域之后，就不再有太多意义了。这个没有那么多意义的额外物质条件，却要用吴的“全部情感需求被忽视，自我完全被扼杀，仅服务于妻子/母亲这两个角色”来换取，这对吴来说是不值得的。两人沟通不了就是慢性自杀，迟早都是要离的，不离的话，只是建立在女人的隐忍基础上。往小了说是情绪价值，其实是漠视了她的付出和独立价值，缺少对她作为人的同理心和尊重。女人生孩子生理上付出的明明那么多，孩子大了想出去读书工作都不被允许……社会对女人结构性压迫让“母亲”面临太多抉择和牺牲，而男人有很多根本无需面对。欣慰的是剧里并没有营造对立，也有李叔徐医生这样的男角色。总之这部剧真的很难得，引起这样的舆论也是有意思的。站在社会既有结构的立场上批评女性……你确实没有看这部剧的必要，因为这部剧所谓的不合当下社会现状的“悬浮感”恰恰是因为超前而不是空想。真正悬浮就不会着重刻画她的处境了，就会是离婚后高歌猛进秒杀前夫的“大女主”叙事了。这部剧只是把女性最真实的处境刻画出来，她的追求理想如果放在一个男人身上想要实现不可能这么艰难，偏偏就因为她是女人，她要面对所有人的不理解，这个角色剧内剧外的处境就很说明问题。只能说现在的孩子一方面太单纯一方面太现实……老实说，人生如果太现实也有点没意思，还是要有点追求。另外呢，读书工作可以是为了钱，也可以是为了理想；50岁抛下工作再生一个，可以是为了满足自己，也可以是为了满足别人。关键是，男的也好女的也好，每个人对自己的事情都有选择的权利，而不是所有人都要按照既定的规则或者是主流的价值观走。你是一点都没有明白生活是一个动词，生活是自己的是与家人共享的，是奔波劳碌是创造的，最重要的是自己选择的。而不是做什么都是带有目的性的：读书是为了找工作，找工作是为了赚钱，赚钱是为了养家，家里有一个人赚钱，另外一个人就呆在家服务家庭。有这种单线逻辑你就永远不会明白吴丽梅为什么要【出走】"
label_review(text, '女人独立')

'[  \n  {  \n    "id": 1,  \n    "一级": "思想精深",  \n    "二级": "主题立意",  \n    "正负向": "正向",  \n    "评论内容": "这部剧只是把女性最真实的处境刻画出来，她的追求理想如果放在一个男人身上想要实现不可能这么艰难，偏偏就因为她是女人，她要面对所有人的不理解，这个角色剧内剧外的处境就很说明问题。"  \n  },  \n  {  \n    "id": 2,  \n    "一级": "思想精深",  \n    "二级": "社会反思",  \n    "正负向": "正向",  \n    "评论内容": "社会对女人结构性压迫让“母亲”面临太多抉择和牺牲，而男人有很多根本无需面对。"  \n  },  \n  {  \n    "id": 3,  \n    "一级": "思想精深",  \n    "二级": "价值导向",  \n    "正负向": "正向",  \n    "评论内容": "关键是，男的也好女的也好，每个人对自己的事情都有选择的权利，而不是所有人都要按照既定的规则或者是主流的价值观走。"  \n  },  \n  {  \n    "id": 4,  \n    "一级": "思想精深",  \n    "二级": "思想深度",  \n    "正负向": "正向",  \n    "评论内容": "你是一点都没有明白生活是一个动词，生活是自己的是与家人共享的，是奔波劳碌是创造的，最重要的是自己选择的。"  \n  },  \n  {  \n    "id": 5,  \n    "一级": "艺术精湛",  \n    "二级": "人物塑造",  \n    "正负向": "正向",  \n    "评论内容": "剧里有表现过她看着职业女性用英文和外宾交流时落寞的样子，她是一个很需要实现自我价值的人，困在家里就算有再多钱对她也是种折磨。"  \n  },  \n  {  \n    "id": 6,  \n    "一级": "艺术精湛",  \n    "二级": "人物塑造",  \n    "正负向": "正向",  \n    "评论内容": "人对生活的需求是多维度的，立体的，除了物质方面得到满足之外，还有精神需求，需要被理解被看到，